In [21]:
from source.util import utils as u
from source import metrics, plots
from source import handshake, handshake2, scargc, hs
import sys
import time
import os
import psutil
import resource
import numpy as np
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score

In [22]:
def doTrain(data, label, step, n_features, percent, epsilon, n_components):

    label_ = label

    size = len(data)
    mod = size % 10
    
#     print(mod)

    if mod != 0:
        data = data[0:(size - mod), :]
        label = label[0:(size - mod)]

    tam = int((size - mod)/10)
#     print(tam)
    # print(tam)
    # sys.exit(0)

    data = np.vsplit(data, 2)
    label = u.chunkify(label, 2)
    label_ = label_[0:(size-mod)]
    
    k = epsilon

    # print(len(data))
    # print(len(label))

    # percent = int(percent)
    # print(percent)
    # print(epsilon)
    acc_percent = []
    f1_percent = []
    mcc_percent = []
    predicted = []
    updt = []
    pool_size = int( len(data[0])/100 * percent )
#     print(pool_size)
    score = 0.0
    f1 = 0.0
    mcc = 0.0
    if pool_size >= k:
        
        pred, updt = scargc.scargc_1NN(data, label_, data[0], label[0], data[1], label[1], pool_size, n_components, n_features, k)
        pred = np.array(pred, dtype=int)
        # print(label[i])
        pred = pred.flatten()
        predicted = pred
        acc_percent, f1_percent, mcc_percent = metrics.makeBatches(label[0], predicted, len(label[0]), step)

        # print(acc_percent, f1_percent, mcc_percent)
        # sys.exit(0)

        for i in range(1, len(data)-1):
            # print(i)
            pred, upd = scargc.scargc_1NN(data, label_, data[i], label[i], data[i+1], label[i+1], pool_size, n_components, n_features, k)
            pred = np.array(pred, dtype=int)
            # print(label[i])
            pred = pred.flatten()

            # print(pred)
            acc, f1, mcc = metrics.makeBatches(label[i], pred, len(label[i]), step)

            predicted = np.vstack([predicted, pred])
            # print(predicted)
            # sys.exit(0)

            updt += upd
            acc_percent = np.vstack([acc_percent, acc])
            f1_percent = np.vstack([f1_percent, f1])
            mcc_percent = np.vstack([mcc_percent, mcc])


        # print(acc_percent)

        predicted = predicted.flatten()
        lab = label_[len(label[0]):]


        score = accuracy_score(lab, predicted)
        f1 = f1_score(lab, predicted, average = 'macro')
        mcc = matthews_corrcoef(lab, predicted)
        # score, f1, mcc = metrics.metrics(acc_percent, lab, predicted, step, f1_type = 'macro')
        # print(a,score, f1, mcc)
        
    return score, f1, mcc

In [23]:
poolsize = 300
clusters = 2
n_components = 4
# episilon = 0.15
step = 5

In [24]:
base = '/home/god/Documentos/tcc/datasets/'
# list = ['NOAA.txt', 'elec.txt']#, 'keystroke.txt']
list = ['keystroke.txt']
# list = ['NOAA.txt']#, 'elec.txt']
database = {}

for i in range(0, len(list)):
    database[i] = base + list[i]

array_ep = [0.05, 0.10, 0.15]
# array_ep = [0.1]
array_p = [10, 20, 30]
# array_p = [30]
array_k = [50, 25, 12, 7, 5, 3, 1]


result = [0.0, 0.0, 0.0]
aux = []
result = np.array(result, dtype=float)

dic = {}

In [25]:
dic = {}
for key, value in database.items():
    dic[key] = {}
    for k in range(0, len(array_k)):
        for p in range(0, len(array_p)):
            adr = value
            dataset, data_labeled, dataset_train, l_train, stream, l_stream, n_features = u.criar_datasets(5, adr)

            score, f1, mcc = doTrain(dataset_train, l_train, step, n_features, array_p[p], array_k[k], n_components)

            aux = np.hstack([score, f1, mcc])
            result = np.vstack([result, aux])

            a = str(array_k[k])
            b = str(array_p[p])
            name = a +' / '+ b
            dic[key][name] = aux
            
            
for p_id, p_info in dic.items():
    print('dataset :', p_id)
    for key in p_info:
        print(key, ':', p_info[key])

/home/god/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/god/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


dataset : 0
50 / 10 : [0. 0. 0.]
50 / 20 : [0. 0. 0.]
50 / 30 : [0. 0. 0.]
25 / 10 : [0. 0. 0.]
25 / 20 : [0. 0. 0.]
25 / 30 : [0. 0. 0.]
12 / 10 : [0. 0. 0.]
12 / 20 : [0. 0. 0.]
12 / 30 : [0.3        0.24623293 0.08691546]
7 / 10 : [0. 0. 0.]
7 / 20 : [0.275      0.21666667 0.03988847]
7 / 30 : [0.55       0.53740309 0.40860313]
5 / 10 : [0. 0. 0.]
5 / 20 : [0.275      0.26314484 0.03651484]
5 / 30 : [0.575      0.57804233 0.45055855]
3 / 10 : [0.25       0.13920455 0.        ]
3 / 20 : [0.375      0.35433723 0.17298035]
3 / 30 : [0.6        0.58567291 0.4742358 ]
1 / 10 : [0.25       0.13920455 0.        ]
1 / 20 : [0.55       0.50487781 0.42559614]
1 / 30 : [0.575      0.56168831 0.44226898]
